# 0 — Introduction to Robotics & ROS2

> Why robots need a software framework, what ROS2 is, and how the ecosystem fits together.

In [ ]:
#| default_exp introduction

## 0.1 What Is a Robot, Really?

At the simplest level a robot is:

```
Sensors  →  Computation  →  Actuators
(camera)     (your code)    (motors)
```

A robot **senses** the world (cameras, LIDAR, IMU, encoders), **processes** that information (obstacle detection, path planning, object recognition), and **acts** on the world (move wheels, grip objects, send signals).

Even a simple robot has dozens of components that need to talk to each other **in real time**. Writing all that communication code from scratch for every project would be a nightmare. That's where **ROS** comes in.

## 0.2 What Is ROS2?

**ROS2** (Robot Operating System 2) is not actually an operating system — it's a **middleware framework** that provides:

| Feature | What it does | Why you care |
|---------|-------------|---------------|
| **Message passing** | Lets independent programs ("nodes") send typed data to each other | You don't write sockets or serialization code |
| **Hardware abstraction** | Standard interfaces for cameras, LIDAR, IMU, motors | Swap hardware without rewriting your logic |
| **Package system** | Thousands of community packages (navigation, SLAM, manipulation) | Don't reinvent the wheel |
| **Build system** (`colcon`) | Compiles C++ and Python packages together | One command builds everything |
| **Launch system** | Start many nodes at once with configuration | Reproducible robot bringup |
| **Visualization** (RViz2) | See sensor data, TF frames, maps in 3D | Debug without guessing |
| **Simulation** (Gazebo) | Test in a physics simulator before real hardware | Safe and cheap iteration |

### ROS1 vs ROS2 — Why ROS2?

| Aspect | ROS1 | ROS2 |
|--------|------|------|
| Architecture | Single master (`roscore`) | **Decentralized** (DDS) — no single point of failure |
| Real-time | Not supported | Designed for **real-time** with DDS QoS |
| Platforms | Linux only | Linux, Windows, macOS, RTOS |
| Security | None built-in | DDS-Security (encryption, auth) |
| Lifecycle | EOL (May 2025) | **Actively maintained** — use this! |

> **Bottom line:** Always start with ROS2. ROS1 is legacy.

## 0.3 The ROS2 Ecosystem at a Glance

```
┌─────────────────────────────────────────────────────────┐
│                    YOUR APPLICATION                     │
│  (Python / C++ nodes: perception, planning, control)    │
├─────────────────────────────────────────────────────────┤
│               ROS2 CLIENT LIBRARIES                     │
│  rclpy (Python)  ·  rclcpp (C++)  ·  rclrs (Rust)       │
├─────────────────────────────────────────────────────────┤
│                ROS2 MIDDLEWARE (RMW)                    │
│  DDS implementations: CycloneDDS · FastDDS · Connext    │
├─────────────────────────────────────────────────────────┤
│                   OPERATING SYSTEM                      │
│  Ubuntu 22.04 / 24.04  (or WSL2 on Windows)             │
└─────────────────────────────────────────────────────────┘
```

Key layers:

- **DDS (Data Distribution Service):** The real-time publish/subscribe protocol underneath ROS2. You rarely touch it directly, but it's what makes ROS2 decentralized and real-time capable.
- **rclpy / rclcpp:** The Python / C++ libraries you import to create nodes, publishers, subscribers, services, etc.
- **colcon:** The build tool (like `make` but for ROS2 packages).
- **ament:** The underlying build system that colcon orchestrates.

## 0.4 Key ROS2 Distributions

ROS2 releases are named alphabetically, like Ubuntu:

| Distribution | Release | EOL | Ubuntu | Status |
|-------------|---------|-----|--------|--------|
| **Humble Hawksbill** | May 2022 | May 2027 | 22.04 | ✅ LTS — recommended for production |
| **Iron Irwini** | May 2023 | Nov 2024 | 22.04 | ❌ EOL |
| **Jazzy Jalisco** | May 2024 | May 2029 | 24.04 | ✅ LTS — newest stable |
| **Rolling** | Continuous | — | Latest | 🔄 Bleeding edge |

> **Recommendation:** Use **Humble** if you need maximum community support, or **Jazzy** if you want the latest Ubuntu 24.04 features. This course works with both.

## 0.5 ROS2 Communication Patterns — A Preview

ROS2 has four main communication patterns. We'll deep-dive in Notebook 02, but here's a taste:

### Topics (Publish / Subscribe)
```
Camera Node  ──/camera/image──►  Processing Node
             (sensor_msgs/Image)
```
- **Asynchronous**, one-to-many
- Best for: continuous data streams (images, laser scans, odometry)

### Services (Request / Response)
```
Client Node  ──request──►  Server Node
             ◄──response──
```
- **Synchronous**, one-to-one
- Best for: quick queries ("is the gripper open?")

### Actions (Goal / Feedback / Result)
```
Client  ──goal──►  Action Server
        ◄─feedback─  (progress %)
        ◄──result──  (done!)
```
- **Asynchronous with feedback**, cancellable
- Best for: long-running tasks ("navigate to point B")

### Parameters
- Key-value configuration that nodes expose at runtime
- Best for: tuning ("set max_speed to 1.5")

## 0.6 Typical Robot Software Architecture

Here's how a camera-based robot might look in ROS2:

```
┌──────────┐    /camera/image    ┌───────────────┐   /detections  ┌──────────┐
│  Camera  │──────────────────►  │  CV / DL Node │───────────────►│ Planner  │
│  Driver  │                     │  (OpenCV+YOLO)│                │  Node    │
└──────────┘                     └───────────────┘                └────┬─────┘
                                                                       │
                                                                   /cmd_vel
                                                                       │
                                                                  ┌────▼─────┐
                                                                  │  Motor   │
                                                                  │  Driver  │
                                                                  └──────────┘
```

Each box is a **node** (an independent process). The arrows are **topics**. This modular design means you can:

1. **Swap** the camera driver without touching the CV node
2. **Test** the planner with fake data (no real hardware needed)
3. **Reuse** the CV node across different robots
4. **Run** nodes on different machines on the same network

## 0.7 What You'll Build in This Course

By the end of the 9 notebooks you will have:

- ✅ A working ROS2 installation (Humble or Jazzy)
- ✅ A properly structured ROS2 workspace with custom packages
- ✅ Python nodes that communicate over topics and services
- ✅ A camera pipeline: raw image → OpenCV processing → annotated output
- ✅ A deep learning node running object detection on a live camera stream

All source code lives in these notebooks and can be exported with `nbdev_export`.

## 0.8 Glossary — Terms You'll See Everywhere

| Term | Meaning |
|------|--------|
| **Node** | A single-purpose process (e.g., camera driver, object detector) |
| **Topic** | A named bus that nodes publish/subscribe to (e.g., `/camera/image`) |
| **Message** | A typed data structure sent over a topic (e.g., `sensor_msgs/msg/Image`) |
| **Package** | A folder with code, config, and a build description — the unit of reuse |
| **Workspace** | A directory containing one or more packages |
| **colcon** | The build tool                                              |
| **rclpy** | The ROS2 Python client library |
| **DDS** | Data Distribution Service — the middleware protocol |
| **QoS** | Quality of Service — reliability/latency settings for topics |
| **TF2** | The transform library — tracks coordinate frames |
| **RViz2** | 3D visualization tool |
| **Gazebo** | Physics simulator |
| **URDF** | Unified Robot Description Format — XML describing a robot's geometry |
| **Launch file** | Python/XML/YAML script that starts multiple nodes |
| **cv_bridge** | Converts between ROS Image messages and OpenCV arrays |

---

**Next →** [Notebook 01: Installing ROS2 & Tools](01_installation.ipynb)